# 1. Import Libary 

In [1]:
import pandas as pd
import pyodbc
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier #KNN
import numpy as np

# 2. Import Data Frame From SQL SERVER

In [2]:
conn = pyodbc.connect( #CONNECTION
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-OQLFA92\\SQLEXPRESS;"
    "DATABASE=vti_dataset;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;")

df_knn = pd.read_sql_query('''
                                        
SELECT *
FROM [vti_dataset].[dbo].[1.Mall_Customers_KNN]
                                       
    ; ''', conn)
df_knn


C:\Users\Admin-s\AppData\Local\Temp\ipykernel_9356\1613490100.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_knn = pd.read_sql_query('''


,CustomerID,Gender,Age,Annual_Income_k,Spending_Score_1_100
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40
...,...,...,...,...,...
195,196,Female,35,120,79
196,197,Female,45,126,28
197,198,Male,32,126,74
198,199,Male,32,137,18


In [4]:
df_knn_new_2026 = df_knn.tail(20)
df_knn_new_2026.to_csv('df_knn_new_2026.csv')

In [6]:
df_knn['label'] = np.random.randint(1, 3, df_knn.shape[0])
df_knn = df_knn.head(180)
df_knn.to_csv("df_knn_old_2025.csv")

C:\Users\Admin-s\AppData\Local\Temp\ipykernel_9356\3679936667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_knn['label'] = np.random.randint(1, 3, df_knn.shape[0])


In [7]:
conn = pyodbc.connect( #CONNECTION
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-OQLFA92\\SQLEXPRESS;"
    "DATABASE=vti_dataset;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;")

df_knn = pd.read_sql_query('''
                                        
SELECT *
FROM [vti_dataset].[dbo].[df_knn_old_2025]
                                       
    ; ''', conn)
df_knn


C:\Users\Admin-s\AppData\Local\Temp\ipykernel_9356\3948480219.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_knn = pd.read_sql_query('''


,column1,CustomerID,Gender,Age,Annual_Income_k,Spending_Score_1_100,label
0,0,1,Male,19,15,39,1
1,1,2,Male,21,15,81,2
2,2,3,Female,20,16,6,1
3,3,4,Female,23,16,77,1
4,4,5,Female,31,17,40,2
...,...,...,...,...,...,...,...
175,175,176,Female,30,88,86,1
176,176,177,Male,58,88,15,2
177,177,178,Male,27,88,69,2
178,178,179,Male,59,93,14,1


# 3. CLEARNING DATA PROCESS

## 3.1 Remove Duplicate Rows

In [8]:
df_knn.duplicated().sum()

np.int64(0)

## 3.2 Detect & Remove Outliers Using IQR

In [9]:
df_knn.describe()
#Pls support me to detect

,column1,CustomerID,Age,Annual_Income_k,Spending_Score_1_100,label
count,180.000000,180.000000,180.000000,180.00000,180.000000,180.000000
mean,89.500000,90.500000,39.100000,55.10000,50.000000,1.483333
std,52.105662,52.105662,14.528377,21.16762,25.133609,0.501116
min,0.000000,1.000000,18.000000,15.00000,1.000000,1.000000
25%,44.750000,45.750000,27.000000,39.00000,35.000000,1.000000
50%,89.500000,90.500000,36.000000,58.50000,50.000000,1.000000
75%,134.250000,135.250000,49.000000,73.00000,71.250000,2.000000
max,179.000000,180.000000,70.000000,93.00000,99.000000,2.000000


## 3.3 Remove Irrelevant Data Using Biz Sense

In [10]:
df_knn.columns

Index(['column1', 'CustomerID', 'Gender', 'Age', 'Annual_Income_k',
       'Spending_Score_1_100', 'label'],
      dtype='object')

## 3.4 Standardization Capitalization

In [11]:
df_knn = df_knn.rename(columns={'Annual_Income_k':'Annual_Income','Spending_Score_1_100':'Spending_Score'})

In [12]:
df_knn

,column1,CustomerID,Gender,Age,Annual_Income,Spending_Score,label
0,0,1,Male,19,15,39,1
1,1,2,Male,21,15,81,2
2,2,3,Female,20,16,6,1
3,3,4,Female,23,16,77,1
4,4,5,Female,31,17,40,2
...,...,...,...,...,...,...,...
175,175,176,Female,30,88,86,1
176,176,177,Male,58,88,15,2
177,177,178,Male,27,88,69,2
178,178,179,Male,59,93,14,1


## 3.5 Covert Data Type

In [13]:
df_knn['Gender'] = np.where(df_knn['Gender']=='Male', 0, 1)

In [14]:
df_knn['Gender']

0      0
1      0
2      1
3      1
4      1
      ..
175    1
176    0
177    0
178    0
179    0
Name: Gender, Length: 180, dtype: int64

## 3.6 Clear Format

## 3.7 Fix Error

## 3.9 Languages Transaltion

## 3.10 Handle Missing Value

# 4. NORMALIZATION DATA

In [18]:
df_knn

,column1,Gender,Age,Annual_Income,Spending_Score,label
0,0,0,19,15,39,1
1,1,0,21,15,81,2
2,2,1,20,16,6,1
3,3,1,23,16,77,1
4,4,1,31,17,40,2
...,...,...,...,...,...,...
175,175,1,30,88,86,1
176,176,0,58,88,15,2
177,177,0,27,88,69,2
178,178,0,59,93,14,1


In [ ]:
df_knn = df_knn.drop(columns=[['column1','CustomerID']])
df_knn

,Gender,Age,Annual_Income,Spending_Score,label
0,0,19,15,39,1
1,0,21,15,81,2
2,1,20,16,6,1
3,1,23,16,77,1
4,1,31,17,40,2
...,...,...,...,...,...
175,1,30,88,86,1
176,0,58,88,15,2
177,0,27,88,69,2
178,0,59,93,14,1


In [23]:
df_knn_label = df_knn
df_knn = df_knn[["Gender","Age","Annual_Income","Spending_Score"]]

In [24]:
scaler = MinMaxScaler()
X = scaler.fit_transform(df_knn[["Gender","Age","Annual_Income","Spending_Score"]])
X_new = scaler.transform(df_knn)
X_new

array([[0.        , 0.01923077, 0.        , 0.3877551 ],
       [0.        , 0.05769231, 0.        , 0.81632653],
       [1.        , 0.03846154, 0.01282051, 0.05102041],
       [1.        , 0.09615385, 0.01282051, 0.7755102 ],
       [1.        , 0.25      , 0.02564103, 0.39795918],
       [1.        , 0.07692308, 0.02564103, 0.76530612],
       [1.        , 0.32692308, 0.03846154, 0.05102041],
       [1.        , 0.09615385, 0.03846154, 0.94897959],
       [0.        , 0.88461538, 0.05128205, 0.02040816],
       [1.        , 0.23076923, 0.05128205, 0.7244898 ],
       [0.        , 0.94230769, 0.05128205, 0.13265306],
       [1.        , 0.32692308, 0.05128205, 1.        ],
       [1.        , 0.76923077, 0.06410256, 0.14285714],
       [1.        , 0.11538462, 0.06410256, 0.7755102 ],
       [0.        , 0.36538462, 0.06410256, 0.12244898],
       [0.        , 0.07692308, 0.06410256, 0.79591837],
       [1.        , 0.32692308, 0.07692308, 0.34693878],
       [0.        , 0.03846154,

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

knn.fit(X, df_knn_label["label"])

,n_neighbors,7
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [94]:
df_knn_new_2026 = df_knn_new_2026[["Gender","Age","Annual_Income_k","Spending_Score_1_100"]]
df_knn_new_2026['Gender'] = np.where(df_knn_new_2026['Gender']=='Male', 0, 1)

In [95]:
scaler = MinMaxScaler()
X = scaler.fit_transform( df_knn_new_2026[["Gender","Age","Annual_Income_k","Spending_Score_1_100"]])
X_2026 = scaler.transform(df_knn_new_2026)
X_2026

array([[0.        , 0.34615385, 0.        , 0.26966292],
       [0.        , 0.15384615, 0.        , 0.87640449],
       [0.        , 0.69230769, 0.025     , 0.07865169],
       [0.        , 0.03846154, 0.025     , 0.8988764 ],
       [0.        , 0.5       , 0.05      , 0.34831461],
       [0.        , 0.07692308, 0.05      , 1.        ],
       [0.        , 1.        , 0.1       , 0.17977528],
       [0.        , 0.        , 0.1       , 0.6741573 ],
       [0.        , 0.5       , 0.15      , 0.1011236 ],
       [0.        , 0.30769231, 0.15      , 0.86516854],
       [0.        , 0.23076923, 0.15      , 0.16853933],
       [0.        , 0.15384615, 0.15      , 0.68539326],
       [0.        , 0.19230769, 0.4       , 0.        ],
       [0.        , 0.38461538, 0.4       , 0.93258427],
       [0.        , 0.73076923, 0.575     , 0.08988764],
       [0.        , 0.26923077, 0.575     , 0.79775281],
       [0.        , 0.65384615, 0.725     , 0.2247191 ],
       [0.        , 0.15384615,

In [98]:
pred = knn.predict(X_2026)
df_knn_new_2026['label'] = pred
df_knn_new_2026

,Gender,Age,Annual_Income_k,Spending_Score_1_100,label
180,1,37,97,32,1
181,1,32,97,86,2
182,1,46,98,15,1
183,1,29,98,88,2
184,1,41,99,39,1
185,1,30,99,97,2
186,1,54,101,24,1
187,1,28,101,68,2
188,1,41,103,17,1
189,1,36,103,85,2


# 5. Save Clear Data Into Database To Visiualization ti

In [100]:
import pyodbc

# --- Connect ---
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=DESKTOP-OQLFA92\\SQLEXPRESS;"
    "DATABASE=vti_dataset;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)
cursor = conn.cursor()

# --- Map pandas dtype -> SQL Server type ---
dtype_map = lambda dt: "INT" if pd.api.types.is_integer_dtype(dt) else \
                       "FLOAT" if pd.api.types.is_float_dtype(dt) else \
                       "BIT" if pd.api.types.is_bool_dtype(dt) else \
                       "DATETIME" if pd.api.types.is_datetime64_any_dtype(dt) else \
                       "VARCHAR(255)"

table_name = "use_case_4_KNN_Normalization" #

# --- Drop + Create table ---
cursor.execute(f"IF OBJECT_ID('dbo.{table_name}', 'U') IS NOT NULL DROP TABLE dbo.{table_name}")
cols = ", ".join([f"[{c}] {dtype_map(df_knn_new_2026[c].dtype)}" for c in df_knn_new_2026.columns]) #
cursor.execute(f"CREATE TABLE dbo.{table_name} ({cols})")

# --- Insert rows ---
cursor.fast_executemany = True
cursor.executemany(f"INSERT INTO dbo.{table_name} VALUES ({','.join(['?']*len(df_knn_new_2026.columns))})", df_knn_new_2026.values.tolist()) #
conn.commit()